In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
from pydrake.all import StartMeshcat

# if you're not launching jupyter from the repo root, fix the path:
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

meshcat = StartMeshcat()
print("Meshcat:", meshcat.web_url())

INFO:drake:Meshcat listening for connections at http://localhost:7000


Meshcat: http://localhost:7000


In [10]:
# simulate_two.py
from rocky.build import build_robot_diagram_two
from rocky import utils
import os
import numpy as np

# assume repo_root and meshcat are already defined in your notebook / script
urdf = os.path.join(repo_root, "rocky.urdf")   # version WITHOUT wall_to_world

bundle = build_robot_diagram_two(
    urdf_path=urdf,
    time_step=1e-3,
    gravity_vec=(0., 0., -9.81),
    meshcat=meshcat,
    enemy_target_xyz=(0., 0.0, 1.1),      # 10 cm off the wall, slightly higher
    enemy_target_rpy=(0.0, 0.0, 0.0),        # facing straight out
    enemy_target_radius=0.05,                # 3 cm sphere
    enemy_target_rgba=(0.2, 0.9, 0.2, 1.0),  # green target
)


plant, ctx, sim = bundle.plant, bundle.plant_context, bundle.simulator
ally, enemy = bundle.ally, bundle.enemy

shoulder_angle = np.deg2rad(-45)
elbow_angle = np.deg2rad(-60)


# Pose each arm independently
utils.set_angles_for_instance(plant, ctx, ally,  shoulder_angle, elbow_angle)
utils.set_angles_for_instance(plant, ctx, enemy, 0.0, 0.0)

# Passive sim for both
utils.set_zero_torques(plant, ctx)
sim.AdvanceTo(1.0)


In [14]:
# Get frames
A_EE = plant.GetFrameByName("glove", model_instance=ally)
T    = plant.GetFrameByName("enemy_target", model_instance=enemy)

# World poses
X_W_ee = plant.CalcRelativeTransform(ctx, plant.world_frame(), A_EE)
X_W_t  = plant.CalcRelativeTransform(ctx, plant.world_frame(), T)

# Relative pose from ally EE to target (use for control)
X_ee_t = plant.CalcRelativeTransform(ctx, A_EE, T)

# For just the target point in world:
p_W_t = X_W_t.translation()

print("A: ", X_ee_t)
print("T: ", p_W_t)

A:  RigidTransform(
  R=RotationMatrix([
    [-0.06396951124607304, -7.834005719052397e-18, -0.9979518533631462],
    [1.2246467991473532e-16, -1.0, 0.0],
    [-0.9979518533631462, -1.2221385429243458e-16, 0.06396951124607304],
  ]),
  p=[-0.5044665689877369, 0.0, 1.4513834213054126],
)
T:  [1.5 0.  0.6]
